In [ ]:
%matplotlib widget
import matplotlib
import numpy as np
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import os, re
import ipywidgets as widgets
import random
# import math

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams.update({'legend.fontsize': 16})
plt.rcParams.update({'axes.titlesize': 16})
plt.rc('xtick', labelsize=16) 
plt.rc('ytick', labelsize=16)

LVC_path = '/misc/home/u1220/analysis_jupyter'
import sys
sys.path.remove('/home/u1220/lvd_aniso/jupiter_books') if '/home/u1220/lvd_aniso/jupiter_books' in sys.path else None
sys.path.append(LVC_path) if not LVC_path in sys.path else None

from LVC.LVCcase_v2 import LVCcase
plt.style.use('LVC')

In [ ]:
experiments_dir = '/misc/home/u1220/lvd_aniso/mol_a_LR/results_LR/id_40'

dirs = os.listdir(experiments_dir)
dirs = [x for x in dirs if x != 'source' and x != 'sources' and os.path.isdir("%s/%s" % (experiments_dir, x))]
# dirs = filter(lambda x: int(re.search(r'verapamil=0.(\d+)$', x).group(1)) == 5, dirs)
# dirs = filter(lambda x: int(re.search(r'Diffuz2=0\.(\d+)', x).group(1)) == 154, dirs)
# dirs = sorted(dirs, key=lambda x: int(re.search('(\d+)_', x).group(1)))
# dirs = sorted(dirs, key=lambda x: int(re.search('u_low=(-\d+)', x).group(1)))
dirs = sorted(dirs, key=lambda x: int(re.search('Ttarget=(\d+)', x).group(1)))

plt.close('all')

for ex_dir in dirs:
    if ex_dir != 'source' and ex_dir != 'sources' and os.path.isdir("%s/%s" % (experiments_dir, ex_dir)):
        edir = ex_dir
        ex_dir = "%s/%s" % (experiments_dir, ex_dir)
        proj = LVCcase(project_folder=ex_dir, auto_load=True, plot=plt)
        proj.read_input(verbose=False)
        
        print(ex_dir)
        ex_dir = ex_dir.replace("_",'\_')
        
        try:           
#             fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
#             time=int(proj.conf['pace_start']) + int(proj.conf['pace_period']) + 1400
# #             time=int(proj.conf['pace_start']) + 60
#             proj.draw_frame(plot=plt, N=int(time/proj.dT), show_tips=True, set_title=True)

            fig = figure(num=None, figsize=(14, 4), dpi=80, facecolor='w')
            proj.draw_tipXY(show_stim=False)
            plt.title('Spiral wave tip X and Y coordinates. %s' % ex_dir[-30:])

            x, y = (90,50)

            x_ind = int(x/(proj.mult_x*proj.drx))
            y_ind = int(y/(proj.mult_y*proj.dry))
            start_t, end_t, u_level = (1000, 60000, -60)
            start_t_ind, end_t_ind = int(start_t/proj.dT), int(end_t/proj.dT)

            times, periods = proj.calculate_periods(x=x_ind, y=y_ind, start=start_t_ind, u_level=u_level)

            figure(num=None, figsize=(12, 4), dpi=80, facecolor='w')
            plt.scatter(times, periods, s=7)
            plt.grid()
            plt.title("Periods at point (%.2f, %.2f) on interval [%.2f, %.2f]. Potential level = %.2f." % 
                      (x_ind*proj.drx*proj.mult_x, y_ind*proj.dry*proj.mult_y, start_t_ind*proj.dT, end_t, u_level))
            plt.show()
            print(np.median(periods))
            
            frame_fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
            time=proj.calc_time - 1000
            proj.draw_frame(plot=plt, N=int(time/proj.dT), show_tips=True, set_title=True, vmax=25)

#             x, y = (30,25)

#             x_ind = int(x/(proj.mult_x*proj.drx))
#             y_ind = int(y/(proj.mult_y*proj.dry))

#             proj.load_udat()
#             time_axis = np.arange(0, proj.frames_number) * proj.dT

#             potential_fig = figure(num=None, figsize=(14, 4), dpi=80, facecolor='w')
#             plt.plot(list(time_axis), list(proj.get_point_in_time(x_ind,y_ind)), linewidth=1, color='red')
#             plt.title('Potential at point (%.2f, %.2f), indices (%d, %d)' % 
#                       (x_ind * proj.drx * proj.mult_x, y_ind * proj.dry * proj.mult_y, x_ind, y_ind))
#             plt.grid(True)
#             plt.xlim(10000,15000)

            plt.show()

            text = widgets.HTML(
                value="<b>%s</b>" % proj.project_folder,
#                 placeholder='Some HTML',
#                 description='Some HTML',
            )
            button = widgets.Button(description='Send this case', tooltip=proj.project_folder)
            def on_button_clicked(self, *args):
                global proj_for_trade
                proj_for_trade = self.tooltip
                %store proj_for_trade
#                 text.value = "Project %s has been stored" % proj_for_trade
            button.on_click(on_button_clicked)
            display(button)
            display(text)
            
#             plt.savefig("sl07/%s" % edir)

        except Exception as e:
            print(e)
#             raise e